In [5]:
import numpy as np
from tensorflow.keras.optimizers import Adam
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
#import tensorflow.keras.applications.ResNet50
#from tensorflow.keras_applications.ResNet import ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.resnet import ResNet50
from tensorflow.keras.models import load_model
import os

 
norm_size = 100
datapath = 'C:\\Users\\HT\\Desktop\\论文\\Back\\data\\train'
EPOCHS = 100
INIT_LR = 1e-3
labelList = []
dicClass = {'cat': 0, 'dog': 1}
classnum = 2
batch_size = 16
def loadImageData():
    imageList = []
    listImage = os.listdir(datapath)

    for img in listImage:
        #labelName = [img.split('.')[0]]
        labelName = dicClass[img.split('.')[0]]
        #print(labelName)
        labelList.append(labelName)
        dataImgPath = os.path.join(datapath, img)
        #print(dataImgPath)
        image = cv2.imdecode(np.fromfile(dataImgPath, dtype=np.uint8), -1)
        image = cv2.resize(image, (norm_size, norm_size), interpolation=cv2.INTER_LANCZOS4)
        image = img_to_array(image)
        imageList.append(image)
    imageList = np.array(imageList, dtype="int") / 255.0
    return imageList
 
 
print("开始加载数据")
imageArr = loadImageData()
labelList = np.array(labelList)
print("加载数据完成")
print(labelList)
trainX, valX, trainY, valY = train_test_split(imageArr, labelList, test_size=0.2, random_state=42)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(featurewise_center=True,
                featurewise_std_normalization=True,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True)
val_datagen = ImageDataGenerator() # 验证集不做图片增强
train_generator = train_datagen.flow(trainX, trainY, batch_size=batch_size, shuffle=True)
val_generator = val_datagen.flow(valX, valY, batch_size=batch_size, shuffle=True)
checkpointer = ModelCheckpoint(filepath='weights_best_Reset50_model.hdf5',
               monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
 
reduce = ReduceLROnPlateau(monitor='val_accuracy', patience=10,
               verbose=1,
               factor=0.5,
               min_lr=1e-6)
 
model = ResNet50(weights=None, classes=classnum)
optimizer = Adam(learning_rate=INIT_LR)
#model= load_model("my_model_resnet.h5")
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_generator,
              steps_per_epoch=trainX.shape[0] / batch_size,
              validation_data=val_generator,
              epochs=EPOCHS,
              validation_steps=valX.shape[0] / batch_size,
              callbacks=[checkpointer, reduce],
              verbose=1, shuffle=True)
model.save('my_model_resnet.h5')

print(history)
 
loss_trend_graph_path = r"WW_loss.jpg"
acc_trend_graph_path = r"WW_acc.jpg"
import matplotlib.pyplot as plt
 
print("Now,we start drawing the loss and acc trends graph...")
# summarize history for accuracy
fig = plt.figure(1)
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.savefig(acc_trend_graph_path)
plt.close(1)
# summarize history for loss
fig = plt.figure(2)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.savefig(loss_trend_graph_path)
plt.close(2)
print("We are done, everything seems OK...")
# #windows系统设置10关机
#os.system("shutdown -s -t 10")
 

开始加载数据
加载数据完成
[0 0 0 ... 1 1 1]


c:\users\ht\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
c:\users\ht\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
c:\users\ht\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/100
100/100 [==============================] - 436s 4s/step - loss: 1.2690 - accuracy: 0.5362 - val_loss: 0.7852 - val_accuracy: 0.4975

Epoch 00001: val_accuracy improved from -inf to 0.49750, saving model to weights_best_Reset50_model.hdf5


c:\users\ht\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
100/100 [==============================] - 440s 4s/step - loss: 0.7779 - accuracy: 0.6025 - val_loss: 1.1463 - val_accuracy: 0.4975

Epoch 00002: val_accuracy did not improve from 0.49750
Epoch 3/100
100/100 [==============================] - 443s 4s/step - loss: 0.7322 - accuracy: 0.6144 - val_loss: 1.0323 - val_accuracy: 0.4975

Epoch 00003: val_accuracy did not improve from 0.49750
Epoch 4/100
100/100 [==============================] - 468s 5s/step - loss: 0.6860 - accuracy: 0.6338 - val_loss: 0.8398 - val_accuracy: 0.4925

Epoch 00004: val_accuracy did not improve from 0.49750
Epoch 5/100
100/100 [==============================] - 472s 5s/step - loss: 0.6691 - accuracy: 0.6356 - val_loss: 0.7987 - val_accuracy: 0.5650

Epoch 00005: val_accuracy improved from 0.49750 to 0.56500, saving model to weights_best_Reset50_model.hdf5
Epoch 6/100
100/100 [==============================] - 479s 5s/step - loss: 0.6441 - accuracy: 0.6413 - val_loss: 0.6449 - val_accuracy: 0.6300

Ep

100/100 [==============================] - 426s 4s/step - loss: 0.5463 - accuracy: 0.7212 - val_loss: 0.7382 - val_accuracy: 0.5925

Epoch 00041: val_accuracy did not improve from 0.67500
Epoch 42/100
100/100 [==============================] - 426s 4s/step - loss: 0.5436 - accuracy: 0.7344 - val_loss: 0.8248 - val_accuracy: 0.5975

Epoch 00042: val_accuracy did not improve from 0.67500

Epoch 00042: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 43/100
100/100 [==============================] - 429s 4s/step - loss: 0.5244 - accuracy: 0.7250 - val_loss: 0.7230 - val_accuracy: 0.6375

Epoch 00043: val_accuracy did not improve from 0.67500
Epoch 44/100
100/100 [==============================] - 451s 5s/step - loss: 0.5088 - accuracy: 0.7469 - val_loss: 0.5961 - val_accuracy: 0.6950

Epoch 00044: val_accuracy improved from 0.67500 to 0.69500, saving model to weights_best_Reset50_model.hdf5
Epoch 45/100
100/100 [==============================] - 438s 4s/step - loss


Epoch 00079: val_accuracy did not improve from 0.77500
Epoch 80/100
100/100 [==============================] - 431s 4s/step - loss: 0.2839 - accuracy: 0.8769 - val_loss: 0.5818 - val_accuracy: 0.7525

Epoch 00080: val_accuracy did not improve from 0.77500
Epoch 81/100
100/100 [==============================] - 431s 4s/step - loss: 0.2757 - accuracy: 0.8888 - val_loss: 0.6629 - val_accuracy: 0.7400

Epoch 00081: val_accuracy did not improve from 0.77500
Epoch 82/100
100/100 [==============================] - 430s 4s/step - loss: 0.2836 - accuracy: 0.8756 - val_loss: 0.7152 - val_accuracy: 0.7025

Epoch 00082: val_accuracy did not improve from 0.77500

Epoch 00082: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 83/100
100/100 [==============================] - 432s 4s/step - loss: 0.2735 - accuracy: 0.8906 - val_loss: 0.5803 - val_accuracy: 0.7925

Epoch 00083: val_accuracy improved from 0.77500 to 0.79250, saving model to weights_best_Reset50_model.hdf5
Epoch 8